In [75]:
import pandas as pd 

df = pd.read_csv("/home/dikidwidasa/mlflow/data/customer.csv")
df = df[df['trans_count'] >0]
df = df.drop(['cus_id'], axis=1)    
len(df)/2


26135.0

In [80]:
bin_labels = [f"{i+1}" for i in range(5)]
df['avg_bal_bins'] = pd.cut(df['avg_bal'], bins=5, labels=bin_labels)
df['trans_count_bin'] = pd.cut(df['trans_count'], bins=5, labels=bin_labels)

df1 = df.iloc[:26135]
df2 = df.iloc[26135:]
df1.to_csv("/home/dikidwidasa/mlflow/data/customer1.csv")
df2.to_csv("/home/dikidwidasa/mlflow/data/customer2.csv")






In [85]:
from scipy.stats import chi2_contingency
import pandas as pd

# Example bin counts
df1_counts = df1['avg_bal_bins'].value_counts().sort_index()
df2_counts = df2['avg_bal_bins'].value_counts().sort_index()
print(
    f"df1_counts: {df1_counts}\n",
    f"df2_counts: {df2_counts}"
)

# Combine counts to ensure all bins are represented
all_bins = set(df1_counts.index).union(df2_counts.index)
df1_counts = df1_counts.reindex(all_bins, fill_value=0)
df2_counts = df2_counts.reindex(all_bins, fill_value=0)

# Add a small constant to avoid zeros (Laplace smoothing)
epsilon = 1e-6
df1_counts += epsilon
df2_counts += epsilon

# Create contingency table
cont_table = pd.DataFrame({'reff': df1_counts, 'test': df2_counts})

# Perform Chi-Square Test
chi2, pvalue, _, _ = chi2_contingency(cont_table.T)
print("Chi-Square Statistic:", chi2)
print("P-value:", pvalue)

if pvalue < 0.05:
    print("Significant drift detected!")
else:
    print("No significant drift detected.")


df1_counts: avg_bal_bins
1        4
2    26128
3        2
4        0
5        1
Name: count, dtype: int64
 df2_counts: avg_bal_bins
1        2
2    26132
3        1
4        0
5        0
Name: count, dtype: int64
Chi-Square Statistic: 2.0003037170599574
P-value: 0.7357030167119538
No significant drift detected.


In [87]:
import whylogs as why
target = why.log(df2).profile().view()
ref = why.log(df1).profile().view()

In [88]:
from whylogs.viz.drift.column_drift_algorithms import calculate_drift_scores

scores = calculate_drift_scores(target_view=target, reference_view=ref, with_thresholds = True)
scores

{'gender': {'algorithm': 'chi-square',
  'pvalue': np.float64(0.1352999880552461),
  'statistic': 2.230614951458122,
  'thresholds': {'NO_DRIFT': (0.15, 1),
   'POSSIBLE_DRIFT': (0.05, 0.15),
   'DRIFT': (0, 0.05)},
  'drift_category': 'POSSIBLE_DRIFT'},
 'marital': {'algorithm': 'ks',
  'pvalue': np.float64(1.0),
  'statistic': 0.0012626745743256063,
  'thresholds': {'NO_DRIFT': (0.15, 1),
   'POSSIBLE_DRIFT': (0.05, 0.15),
   'DRIFT': (0, 0.05)},
  'drift_category': 'NO_DRIFT'},
 'generation': {'algorithm': 'chi-square',
  'pvalue': np.float64(0.0963816451235561),
  'statistic': 4.678878997554513,
  'thresholds': {'NO_DRIFT': (0.15, 1),
   'POSSIBLE_DRIFT': (0.05, 0.15),
   'DRIFT': (0, 0.05)},
  'drift_category': 'POSSIBLE_DRIFT'},
 'own_house': {'algorithm': 'chi-square',
  'pvalue': np.float64(0.9283295745441676),
  'statistic': 0.008090407878132871,
  'thresholds': {'NO_DRIFT': (0.15, 1),
   'POSSIBLE_DRIFT': (0.05, 0.15),
   'DRIFT': (0, 0.05)},
  'drift_category': 'NO_DRIFT'},


In [81]:
df.columns

Index(['gender', 'marital', 'generation', 'own_house', 'education', 'job',
       'inactive', 'income', 'avg_bal', 'trans_count', 'credit_trans_amount',
       'recency_score', 'avg_bal_bins', 'trans_count_bin'],
      dtype='object')

In [64]:
# Get descriptive statistics
summary_stats = df1.describe()

# Format the columns to display in standard decimal format with 2 decimal places
formatted_stats = summary_stats.style.format("{:.2f}")

# Display the formatted statistics
formatted_stats

,marital,income,avg_bal,trans_count,credit_trans_amount,recency_score
count,26135.00,26135.00,26135.00,26135.00,26135.00,26135.00
mean,0.75,1.58,7985220.49,93.98,-97043250.68,4.44
std,0.44,1.47,111974338.28,570.54,1701026020.69,1.23
min,0.00,0.00,-4426324104.09,1.00,-232513447474.00,1.00
25%,0.00,1.00,24062.31,4.00,-26665795.50,5.00
50%,1.00,1.00,139014.61,7.00,-9994695.00,5.00
75%,1.00,1.00,513676.83,11.00,-4943314.00,5.00
max,1.00,6.00,13178211355.55,26185.00,0.00,5.00


In [65]:
# Get descriptive statistics
summary_stats = df2.describe()

# Format the columns to display in standard decimal format with 2 decimal places
formatted_stats = summary_stats.style.format("{:.2f}")

# Display the formatted statistics
formatted_stats

,marital,income,avg_bal,trans_count,credit_trans_amount,recency_score
count,26135.00,26135.00,26135.00,26135.00,26135.00,26135.00
mean,0.74,1.57,8072799.30,214.41,-107705148.48,4.43
std,0.44,1.45,67641656.15,14358.89,3057975164.93,1.24
min,0.00,0.00,-2074429647.72,1.00,-438877675264.00,1.00
25%,0.00,1.00,22712.28,4.00,-26189047.50,5.00
50%,1.00,1.00,140848.04,7.00,-9983600.00,5.00
75%,1.00,1.00,511548.21,11.00,-4858388.00,5.00
max,1.00,6.00,3260283217.11,2163385.00,0.00,5.00


In [66]:
def defineDriftCondition(start : int, end : int, df : pd.DataFrame) -> int:
    df_list = df['trans_count'].tolist()
    drift = []
    for i in df_list:
        if df_list[i] > start and df_list[i] < end:
            drift.append(1)
        else:
            drift.append(0)
    return sum(drift)

def defineDriftCondition2(start: int, end: int, df: pd.DataFrame) -> int:
    drift = ((df['trans_count'] < start)|(df['trans_count'] > end)).sum()
    return (round(drift/len(df)*100))


In [67]:
max = df1['trans_count'].max()
min = df1['trans_count'].min()
print(f"max : {max}, min : {min}")
def defineDriftCondition2(start: int, end: int, df: pd.DataFrame) -> int:
    drift = ((df['trans_count'] < start)|(df['trans_count'] > end)).sum()
    return (round(drift/len(df)*100))
x = defineDriftCondition2(1, max, df2)
x


max : 26185, min : 1


0